In [8]:
using PyPlot
using JuMP, Ipopt
using Combinatorics

In [9]:
include("../src/model.jl");
include("../src/diagnostics.jl");
include("../src/optimization.jl");
include("../src/plotting.jl");
include("../src/defaults.jl");

# Optimizating the Global Climate Action Policy Portfolio

In [10]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>1. /2.)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.5

In [11]:
controls = collect(keys(max_deployment_og));

In [12]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [13]:
results = Dict()

for control_permutation in control_permutations    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0
        end
    end
    
    model = ClimateModel();
    @time m = optimize_controls!(model, max_deployment = max_deployment);
    
    results[control_permutation] = Dict("NPV" => discounted_total_control_cost(model), "status" => raw_status(m))
    
end

Solve_Succeeded
  3.638894 seconds (4.01 M allocations: 198.203 MiB, 2.51% gc time)
Solve_Succeeded
  0.040634 seconds (97.68 k allocations: 4.107 MiB)
Infeasible_Problem_Detected
  1.482047 seconds (2.90 M allocations: 54.146 MiB, 3.01% gc time)
Infeasible_Problem_Detected
  1.142068 seconds (1.24 M allocations: 23.377 MiB)
Solve_Succeeded
  0.051461 seconds (95.76 k allocations: 4.132 MiB)
Solve_Succeeded
  0.036729 seconds (74.92 k allocations: 3.755 MiB)
Solve_Succeeded
  0.038930 seconds (85.05 k allocations: 3.931 MiB)
Solve_Succeeded
  0.063123 seconds (112.65 k allocations: 4.437 MiB)
Solve_Succeeded
  0.096029 seconds (126.42 k allocations: 4.692 MiB, 26.21% gc time)
Solve_Succeeded
  0.074240 seconds (201.53 k allocations: 5.927 MiB)
Solve_Succeeded
  0.042501 seconds (101.00 k allocations: 4.281 MiB)
Solve_Succeeded
  0.041743 seconds (101.31 k allocations: 4.282 MiB)
Solve_Succeeded
  0.033791 seconds (87.53 k allocations: 4.021 MiB)
Solve_Succeeded
  0.053556 seconds (128.

In [14]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


["geoeng", "mitigate", "remove"], 143, 5%
["remove"] has no solution.
["mitigate", "remove"], 188, 38%
["mitigate"], 322, 136%
["geoeng", "mitigate", "remove", "adapt"], 136, 0%
["geoeng", "adapt"], 285, 109%
["geoeng", "mitigate", "adapt"], 183, 34%
["mitigate", "remove", "adapt"], 179, 31%
["geoeng"], 411, 201%
["geoeng", "remove", "adapt"], 222, 63%
["mitigate", "adapt"], 279, 105%
["geoeng", "mitigate"], 199, 46%
["remove", "adapt"], 431, 216%
["geoeng", "remove"], 267, 96%
["adapt"] has no solution.
